In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

False

In [15]:
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT=os.getenv("LANGCHAIN_PROJECT")

In [16]:

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"]= GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]=LANGCHAIN_PROJECT


In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma

In [18]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.0-pro")

In [19]:
'''from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")'''
from langchain_groq import ChatGroq
import os
llm=ChatGroq(model_name="Gemma2-9b-It")

In [20]:
while True:
    question=input("type your question. if you want to quit the chat write quit")
    if question !="quit":
        print(llm.invoke(question).content)
    else:
        print("goodbye take care yourself")
        break

goodbye take care yourself


In [21]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

In [22]:
store={}

In [23]:

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [24]:

config = {"configurable": {"session_id": "firstchat"}}

In [25]:
model_with_memory=RunnableWithMessageHistory(llm,get_session_history)

In [26]:

model_with_memory.invoke(("Hi! I'm sunnysavita"),config=config).content

"Hello Sunnysavita! 👋\n\nIt's nice to meet you. What can I do for you today? 😊  \n\n"

In [27]:
model_with_memory.invoke(("tell me what is my name?"),config=config).content

'Your name is Sunnysavita!  😊 \n\nI remember that from our introduction.  \n\nIs there anything else I can help you with?  ✨\n'

In [35]:
for i in store['firstchat'].messages:
    print(i.content)


Hi! I'm sunnysavita
Hello Sunnysavita! 👋

It's nice to meet you. What can I do for you today? 😊  


tell me what is my name?
Your name is Sunnysavita!  😊 

I remember that from our introduction.  

Is there anything else I can help you with?  ✨



In [39]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough , RunnableLambda
from langchain_core.output_parsers import StrOutputParser

### Reading the txt files from source directory

loader = DirectoryLoader('data', glob="./*.txt", loader_cls=TextLoader)
docs = loader.load()

### Creating Chunks using RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
    length_function=len
)
new_docs = text_splitter.split_documents(documents=docs)
doc_strings = [doc.page_content for doc in new_docs]

###  BGE Embddings

'''from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)
'''

### Creating Retriever using Vector DB

db = Chroma.from_documents(new_docs, embeddings)
retriever = db.as_retriever(search_kwargs={"k": 4})

In [40]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)


In [41]:

retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
    )

In [42]:
question ="what is llama3? can you highlight 3 important points?"
print(retrieval_chain.invoke(question))

Llama 3 is an 8B parameter language model released in April 2024. 

Here are three important points:

* **8B Parameter Version:**  This refers to a specific size of the model, indicating its complexity and capacity.
* **Released in April 2024:** This establishes the timeframe of its availability.
* **Accessible Outside:**  Llama 3 versions are available for use beyond just Meta's own platforms. 


Let me know if you have any other questions! 



# Let's Start with Tools and Agents

In [43]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [67]:
api_wrapper = WikipediaAPIWrapper()

In [68]:
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [46]:
tool.name

'wikipedia'

In [47]:
tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [48]:
tool.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [49]:
tool.return_direct

False

In [69]:
print(tool.run({"query": "langchain"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.

Page: Milvus (vector database)
Summary: Milvus is a distributed vector database developed by Zilliz. It is available as both open-source software and a cloud service.
Milvus is an open-source project under LF AI & Data Foundation distributed under the Apache License 2.0.

Page: Retrieval-augmented generation
Summary: Retrieval-Augmented Generation (RAG) is a technique that grants generative artificial intelligence models information retrieval capabilities. It modifies interactions with a large language model (LLM) so that the model responds to user queries with reference to a specified set of documents, using this information to augment informa

In [70]:
tool.run("langchain")

"Page: LangChain\nSummary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\nPage: Milvus (vector database)\nSummary: Milvus is a distributed vector database developed by Zilliz. It is available as both open-source software and a cloud service.\nMilvus is an open-source project under LF AI & Data Foundation distributed under the Apache License 2.0.\n\nPage: Retrieval-augmented generation\nSummary: Retrieval-Augmented Generation (RAG) is a technique that grants generative artificial intelligence models information retrieval capabilities. It modifies interactions with a large language model (LLM) so that the model responds to user queries with reference to a specified set of documents, using this information to augmen

In [52]:
from langchain_core.pydantic_v1 import BaseModel, Field

e:\pythonProject\Langgraph\venv\Lib\site-packages\IPython\core\interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [53]:
class WikiInputs(BaseModel):
    query: str = Field(description="query to look up in Wikipedia, should be 3 or less words")

In [71]:
from pydantic import BaseModel, Field

class WikiInputs(BaseModel):
    query: str= Field(description="query to look up in Wikipedia, should be 3 or less words")  # Define the required query input

tool = WikipediaQueryRun(
    name="wiki-tool",
    description="Look up things in Wikipedia",
    args_schema=WikiInputs,  # Ensure this is a valid Pydantic model
    api_wrapper=api_wrapper,
    return_direct=True,
)


In [72]:
tool.name

'wiki-tool'

In [73]:
tool.description

'Look up things in Wikipedia'

In [62]:
tool.args

{'query': {'description': 'query to look up in Wikipedia, should be 3 or less words',
  'title': 'Query',
  'type': 'string'}}

In [63]:
tool.return_direct

True

In [74]:
print(tool.run("langchain"))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.

Page: Milvus (vector database)
Summary: Milvus is a distributed vector database developed by Zilliz. It is available as both open-source software and a cloud service.
Milvus is an open-source project under LF AI & Data Foundation distributed under the Apache License 2.0.

Page: Retrieval-augmented generation
Summary: Retrieval-Augmented Generation (RAG) is a technique that grants generative artificial intelligence models information retrieval capabilities. It modifies interactions with a large language model (LLM) so that the model responds to user queries with reference to a specified set of documents, using this information to augment informa

# youtube search tool

In [75]:
from langchain_community.tools import YouTubeSearchTool

In [76]:
tool=YouTubeSearchTool()

In [77]:
tool.name

'youtube_search'

In [78]:
tool.description

'search for youtube videos associated with a person. the input to this tool should be a comma separated list, the first part contains a person name and the second a number that is the maximum number of video results to return aka num_results. the second part is optional'

In [80]:
tool.run("sunny savita")

"['https://www.youtube.com/watch?v=s11yOKNXOQU&pp=ygUMc3Vubnkgc2F2aXRh', 'https://www.youtube.com/watch?v=uQY_7LKS8sc&pp=ygUMc3Vubnkgc2F2aXRh']"

In [82]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [83]:
tool = TavilySearchResults()

In [84]:
tool.invoke({"query": "What happened in the latest burning man floods"})

[{'url': 'https://www.npr.org/2023/09/03/1197497458/the-latest-on-the-burning-man-flooding',
  'content': "There are also reports that at least one person has died at the counterculture festival about a hundred miles north of Reno, Nev. Earlier this afternoon, I caught up with NPR's Claudia Peschiutta, who's at her first burn, and she told me it's muddy where she is, but that she and her camp family have been making the best of things.\n National\nThe latest on the Burning Man flooding\nClaudia Peschiutta\nAuthorities are investigating a death at the Burning Man festival in the Nevada desert after tens of thousands of people are stuck in camps because of rain.\n SCOTT DETROW, HOST:\nKnee-deep mud, warnings to conserve food and water, orders to shelter in place - this is all at Burning Man 2023 after torrential rains turned the Black Rock Desert into miles and miles of mud. I mean, mostly what I've seen from my personal experience is just any sort of need that you have, somebody, whethe

In [46]:
from langchain import hub

In [51]:
from langchain.agents import AgentExecutor,create_openai_functions_agent

In [48]:
instructions = """You are an assistant."""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)

In [63]:
tavily_tool = TavilySearchResults()

In [64]:
tools = [tavily_tool]

In [65]:
agent = create_openai_functions_agent(llm, tools, prompt)

In [66]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)

In [67]:
print(agent_executor.invoke({"input": "who was the mahatma gandhi"}))

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `tavily_search_results_json` with `{'query': 'who was the mahatma mahatma gandhi'}`


[{'url': 'https://historycooperative.org/stay-story-mahatma-gandhi/', 'content': "Mahatma Gandhi, also known as Mohandas Karamchand Gandhi, was a prominent figure in India's struggle for independence from British rule. His approach to non-violent protest and civil disobedience became a beacon for peaceful movements worldwide. Gandhi's beliefs in simplicity, non-violence, and truth had a profound impact on the world, influencing other leaders like Martin Luther"}, {'url': 'https://www.britannica.com/summary/Mahatma-Gandhi', 'content': 'Mahatma Gandhi summary\nExplore the life of Mahatma Gandhi as a political and social activist\nMahatma Gandhi, byname of Mohandas Karamchand Gandhi,\n(born Oct. 2, 1869, Porbandar, India—died Jan. 30, 1948, Delhi), Preeminent leader of Indian nationalism and prophet of nonviolence in the 20th century.\n He refashioned the Indian National Congress into an effec

# Showcase that one more agent also

# Create our custom agent and custom tools

In [69]:
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from pydantic import BaseModel

In [71]:
@tool
def search(query: str) -> str:
    """Look up things online."""
    return "LangChain"

In [72]:
print(search.name)
print(search.description)
print(search.args)

search
Look up things online.
{'query': {'title': 'Query', 'type': 'string'}}


In [73]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [74]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [76]:
class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")

In [77]:

@tool("search-tool", args_schema=SearchInput, return_direct=True)
def search(query: str) -> str:
    """Look up things online."""
    return "LangChain"

In [78]:
print(search.name)
print(search.description)
print(search.args)
print(search.return_direct)

search-tool
Look up things online.
{'query': {'title': 'Query', 'type': 'string'}}
True


c:\Users\sunny\LangGraph-End-to-End-Course\venv\lib\site-packages\pydantic\json_schema.py:2191: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='should be a search query' extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


In [ ]:
from typing import Optional, Type

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

In [79]:
class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")

In [80]:
class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")

# here is my custom tool

In [82]:
from langchain.agents import tool
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

In [83]:
get_word_length.invoke("abc")

3

In [84]:
tools = [get_word_length]

In [85]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [86]:
llm_with_tools = llm.bind_tools(tools)

In [88]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser



In [89]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [90]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [91]:
list(agent_executor.stream({"input": "How many letters in the word eudca"}))



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'eudca'}`


55

> Finished chain.


[{'actions': [ToolAgentAction(tool='get_word_length', tool_input={'word': 'eudca'}, log="\nInvoking: `get_word_length` with `{'word': 'eudca'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_4188', 'function': {'arguments': '{"word":"eudca"}', 'name': 'get_word_length'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-6cc153ac-50f7-468e-871a-bb81fd15b1f7', tool_calls=[{'name': 'get_word_length', 'args': {'word': 'eudca'}, 'id': 'call_4188', 'type': 'tool_call'}], usage_metadata={'input_tokens': 966, 'output_tokens': 86, 'total_tokens': 1052}, tool_call_chunks=[{'name': 'get_word_length', 'args': '{"word":"eudca"}', 'id': 'call_4188', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_4188')],
  'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_4188', 'function': {'arguments': '{"word":"eudca"}', 'name': 'get_word_length'}, 'type': 'fu

[{'actions': [ToolAgentAction(tool='get_word_length', tool_input={'word': 'eudca'}, log="\nInvoking: `get_word_length` with `{'word': 'eudca'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_4188', 'function': {'arguments': '{"word":"eudca"}', 'name': 'get_word_length'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-6cc153ac-50f7-468e-871a-bb81fd15b1f7', tool_calls=[{'name': 'get_word_length', 'args': {'word': 'eudca'}, 'id': 'call_4188', 'type': 'tool_call'}], usage_metadata={'input_tokens': 966, 'output_tokens': 86, 'total_tokens': 1052}, tool_call_chunks=[{'name': 'get_word_length', 'args': '{"word":"eudca"}', 'id': 'call_4188', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_4188')],
  'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_4188', 'function': {'arguments': '{"word":"eudca"}', 'name': 'get_word_length'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-6cc153ac-50f7-468e-871a-bb81fd15b1f7', tool_calls=[{'name': 'get_word_length', 'args': {'word': 'eudca'}, 'id': 'call_4188', 'type': 'tool_call'}], usage_metadata={'input_tokens': 966, 'output_tokens': 86, 'total_tokens': 1052}, tool_call_chunks=[{'name': 'get_word_length', 'args': '{"word":"eudca"}', 'id': 'call_4188', 'index': 0, 'type': 'tool_call_chunk'}])]},
 {'steps': [AgentStep(action=ToolAgentAction(tool='get_word_length', tool_input={'word': 'eudca'}, log="\nInvoking: `get_word_length` with `{'word': 'eudca'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_4188', 'function': {'arguments': '{"word":"eudca"}', 'name': 'get_word_length'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-6cc153ac-50f7-468e-871a-bb81fd15b1f7', tool_calls=[{'name': 'get_word_length', 'args': {'word': 'eudca'}, 'id': 'call_4188', 'type': 'tool_call'}], usage_metadata={'input_tokens': 966, 'output_tokens': 86, 'total_tokens': 1052}, tool_call_chunks=[{'name': 'get_word_length', 'args': '{"word":"eudca"}', 'id': 'call_4188', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_4188'), observation=5)],
  'messages': [FunctionMessage(content='5', additional_kwargs={}, response_metadata={}, name='get_word_length')]},
 {'output': '5',
  'messages': [AIMessage(content='5', additional_kwargs={}, response_metadata={})]}]

In [92]:
llm.invoke("How many letters in the word educa")

AIMessage(content='Here\'s how to solve this:\n\n* **Focus on the phrase:**  The question asks about the letters in the phrase "letters in the word educa".\n* **Count carefully:** Count each letter in that phrase.\n\nLet me know if you\'d like me to do the counting for you! \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 16, 'total_tokens': 83, 'completion_time': 0.121818182, 'prompt_time': 0.00024042, 'queue_time': 0.042495864, 'total_time': 0.122058602}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-ed352c53-84e7-48d6-9576-0acd99befd1f-0', usage_metadata={'input_tokens': 16, 'output_tokens': 67, 'total_tokens': 83})

In [93]:
from langchain_core.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but bad at calculating lengths of words.",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [94]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

In [95]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [96]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [97]:
input1 = "how many letters in the word educa?"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
agent_executor.invoke({"input": "is that a real word?", "chat_history": chat_history})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `get_word_length` with `{'word': 'educa'}`


5

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


5

> Finished chain.
Yes. 


> Finished chain.


{'input': 'is that a real word?',
 'chat_history': [HumanMessage(content='how many letters in the word educa?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='5', additional_kwargs={}, response_metadata={})],
 'output': 'Yes. \n'}